In [76]:
import pandas as pd

all_file_df = pd.read_csv("../classifier.csv")

all_file_df['probability'] = pd.to_numeric(all_file_df['probability'])

threshold = 0.5

positive_df = all_file_df[all_file_df['probability'] > threshold]
negative_df = all_file_df[all_file_df['probability'] <= threshold]

all_file_df

,file_path,target_path,probability,type
0,./data/docx/题库/生物/高中/数学(2)/高中/高考真题【391.82MB】/2...,./data/docx/题库/生物/高中/数学(2)/高中/高考真题【391.82MB】/2...,1.000000e+00,生物
1,./data/docx/题库/生物/高中/jiaocai/教材/学校教育（电子教材）/高中学...,./data/docx/题库/生物/高中/jiaocai/教材/学校教育（电子教材）/高中学...,1.000000e+00,生物
2,./data/docx/题库/生物/初中/初中人教版电子课本/初中人教版电子课本/9--j生...,./data/docx/题库/生物/初中/初中人教版电子课本/初中人教版电子课本/9--j生...,9.999674e-01,生物
3,./data/docx/题库/生物/初中/小品/按合集分/【科教类】/【初中思维导图】_9科...,./data/docx/题库/生物/初中/小品/按合集分/【科教类】/【初中思维导图】_9科...,9.214322e-01,生物
4,./data/docx/题库/数学/小学/其他/小学-1-6年级试卷合集--25.3 GB/...,./data/docx/题库/数学/小学/其他/小学-1-6年级试卷合集--25.3 GB/...,9.214322e-01,语文
...,...,...,...,...
8270,./data/docx/题库/英语/jiaocai/教材/学校教育（电子教材）/教材教辅与参...,./data/docx/题库/英语/jiaocai/教材/学校教育（电子教材）/教材教辅与参...,9.156381e-23,NaN
8271,./data/docx/题库/英语/jiaocai/教材/学校教育（电子教材）/高中学习资料...,./data/docx/题库/英语/jiaocai/教材/学校教育（电子教材）/高中学习资料...,2.600176e-05,NaN
8272,./data/docx/题库/英语/jiaocai/教材/学校教育（电子教材）/高中学习资料...,./data/docx/题库/英语/jiaocai/教材/学校教育（电子教材）/高中学习资料...,5.739219e-02,NaN
8273,./data/docx/题库/英语/jiaocai/教材/学校教育（电子教材）/高中学习资料...,./data/docx/题库/英语/jiaocai/教材/学校教育（电子教材）/高中学习资料...,1.001992e-01,NaN


# 检查类型

In [77]:
positive_df['type'].value_counts()

数学       2047
医学        467
语文        279
生物        248
other     178
化学        173
物理        152
地理        133
历史         70
政治         69
理综         62
文综         62
Name: type, dtype: int64

In [75]:
for _, row in all_file_df.iterrows():
    if row['type'] == "other":
        print(f"{row['type']} {row['file_path']}")

other ./data/docx/题库/数学/小学/其他/小学-1-6年级试卷合集--25.3 GB/3部分/【11.30更新】北师大小学数学1-6年级上下册试题试卷/数学2上 北师大/数学北师大2上试卷 118份/期中期末 5份/北师大版数学二年级上学期期中试卷1/2016-2017学年北师大上学期二年级期中检测卷答案更多资源-XH1080.com.doc
other ./data/docx/题库/数学/小学/其他/小学-1-6年级试卷合集--25.3 GB/3部分/【11.30更新】北师大小学数学1-6年级上下册试题试卷/数学2上 北师大/数学北师大2上试卷 118份/期中期末 5份/北师大版数学二年级上学期期末试卷1/北师大2016-2017学年上学期二年级期末检测卷更多资源-XH1080.com.doc
other ./data/docx/题库/数学/小学/其他/小学-1-6年级试卷合集--25.3 GB/3部分/【11.30更新】北师大小学数学1-6年级上下册试题试卷/数学6下 北师大/数学北师大6下试卷 133份/数学小升初专项训练 55份/小升初知识点复习专项练习-数的运算21小数除法-通用版（2015）更多资源-XH1080.com.doc
other ./data/docx/题库/数学/小学/其他/小学-1-6年级试卷合集--25.3 GB/3部分/【11.30更新】北师大小学数学1-6年级上下册试题试卷/数学6下 北师大/数学北师大6下试卷 133份/数学小升初专项训练 55份/小升初知识点复习专项练习-数的运算06表内乘法-通用版（2015）更多资源-XH1080.com.doc
other ./data/docx/题库/数学/小学/其他/小学-1-6年级试卷合集--25.3 GB/A部分/优教之家：统编版1-6年级语文（上）期末专项突破卷/三年级上册/期末专项突破卷/5说明性文本阅读专项突破卷更多资源-XH1080.com.doc
other ./data/docx/题库/数学/小学/其他/小学-1-6年级试卷合集--25.3 GB/A部分/优教之家：统编版1-6年级语文（上）期末专项突破卷/一年级上册/期末专项突破卷/4积累背诵与课文理解专项突破卷更多资源-XH1080.com.docx
other ./data/docx/题库/数学/小

# 检查文类成试卷是否正确

In [53]:
def query_file_total_by_keywords(df, keywords: list|set, uninclude_keywords: list|set = []):
    # keywords和uninclude_keywords可以被抽象理解成路径，如 "/题库"
    # keywords: 必须全部出现在"file_path"中，才会被统计
    # uninclude_keywords: 只要出现一个，就不会被统计

    # 创建一个全为 True 的布尔系列，它将用于存储是否每个文件路径包含所有关键字
    contains_all_keywords = pd.Series([True]*len(all_file))

    # 检查文件路径是否包含所有关键字
    for keyword in keywords:
        contains_all_keywords = contains_all_keywords & df['file_path'].fillna('').str.contains(keyword)

    if uninclude_keywords:
        # 检查文件路径是否包含 uninclude_keywords 中的任何一个
        for keyword in uninclude_keywords:
            contains_all_keywords = contains_all_keywords & ~df['file_path'].fillna('').str.contains(keyword)

    filtered_df = df.loc[contains_all_keywords]

    return filtered_df

# 检查"题库"文件夹

# 由于"学前教辅资料"占比比较多，所以优先查看

In [58]:
# 题库中所有文件数量
filtered_all_file_df  = query_file_total_by_keywords(all_file_df, {"/题库"})
len(filtered_all_file_df)

3310

In [60]:
# 题库中所有"学前教辅资料"文件数量
specified_keywords_filtered_all_file_df = query_file_total_by_keywords(all_file_df, {"/题库", "/学前教辅资料"})
len(specified_keywords_filtered_all_file_df)

3307

In [61]:
len(specified_keywords_filtered_all_file_df) / len(filtered_all_file_df)

0.9990936555891239

In [63]:
# 从题库中筛选出来的所有文件数量
len(query_file_total_by_keywords(positive_df, {"/题库"}))

6

In [64]:
# 从题库中筛选出来的"学前教辅资料"所有文件(全都是错误的归类)
query_file_total_by_keywords(positive_df, {"/题库", "/学前教辅资料"})

,file_path,target_path,predict,probability
660,./data/docx/题库/数学/小学/其他/教辅资料库 小初高（合集）/学前教辅资料超级...,./data/classifier2/题库/数学/小学/其他/教辅资料库 小初高（合集）/学...,1,0.6312903929689908
1202,./data/docx/题库/数学/小学/其他/教辅资料库 小初高（合集）/学前教辅资料超级...,./data/classifier2/题库/数学/小学/其他/教辅资料库 小初高（合集）/学...,1,0.6106443667548755
1427,./data/docx/题库/数学/小学/其他/教辅资料库 小初高（合集）/学前教辅资料超级...,./data/classifier2/题库/数学/小学/其他/教辅资料库 小初高（合集）/学...,1,0.9597107825314274
2299,./data/docx/题库/数学/小学/其他/教辅资料库 小初高（合集）/学前教辅资料超级...,./data/classifier2/题库/数学/小学/其他/教辅资料库 小初高（合集）/学...,1,0.8557713596029245


# 从文件名(文件路径)推测

In [65]:
# 从文件路径推测题库文件夹中试卷
filtered_all_file_df = query_file_total_by_keywords(all_file_df, {"/题库"}, {"小品", "/学前教辅资料"})
len(filtered_all_file_df)

3

In [66]:
# 从文件路径推测题库中筛选出来的文件夹中试卷
specified_keywords_filtered_positive_file_df = query_file_total_by_keywords(positive_df, {"/题库"}, {"小品", "/学前教辅资料"})
len(specified_keywords_filtered_positive_file_df)

2

In [13]:
# 是"题库"文件夹中的文件，但是预测结果为False
set(examination_all_file_path) - set(examination_positive_file_path)

{'./data/docx/题库/医学/10T电子书 超全/未分类/004 - 菜根阁整站电子书（123G）/04畅销书2万本（pdf txt doc chm）/1.世界顶级畅销书530本（Word）/中医基础理论（新世纪教材）.doc',
 './data/docx/题库/医学/10T电子书 超全/未分类/世界顶级畅销书530本/一个学中医的女生写出的保养大法.doc',
 './data/docx/题库/医学/10T电子书 超全/👑👑各类分类/书藉大合集/中医资料（1.94T）_分享/南京中医药大学学习课件/按摩手法.doc',
 './data/docx/题库/医学/10T电子书 超全/👑👑各类分类/书藉大合集/中医资料（1.94T）_分享/南京中医药大学学习课件/推拿学基础/9辅助方法.doc',
 './data/docx/题库/医学/10T电子书 超全/👑👑各类分类/书藉大合集/中医资料（1.94T）_分享/经方系列/傷寒論全部藥方分類錄.doc',
 './data/docx/题库/医学/10T电子书 超全/👑👑各类分类/书藉大合集/中医资料（1.94T）_分享/食疗/生活百科：盘点冬季萝卜食疗效果最好的八种吃法 .doc',
 './data/docx/题库/医学/10T电子书 超全/👑👑各类分类/书藉大合集/中医资料（1.94T）_分享/食疗/茶/冬季养生：吃羊肉五大禁忌 忌与茶同食.doc',
 './data/docx/题库/医学/10T电子书 超全/👑👑各类分类/书藉大合集/中医资料（1.94T）_分享/食疗/茶/生姜红茶 为冬季减肥做准备 .doc',
 './data/docx/题库/医学/10T电子书 超全/👑👑各类分类/书藉大合集/中医资料（1.94T）_分享/食疗/酒/冬季吃火锅喝烈酒悠着点 葡萄酒养生避四大雷区.doc',
 './data/docx/题库/医学/10T电子书 超全/👑👑各类分类/书藉大合集/中医资料（1.94T）_分享/食疗/酒/年底聚会多 男人如何健康饮酒 .doc',
 './data/docx/题库/医学/10T电子书 超全/👑👑各类分类/书藉大合集/低幼到小学绘本1088本/112本绘本集+112教案/教案/鳄鱼怕怕牙医怕怕.doc',
 './data/docx/题库/医学/Y 医学资料合集（包含200

# 在全部的范围中使用文件名推测

In [14]:
def query_file_total_by_any_keywords(all_file, keywords, uninclude_keywords=[]):
    total = 0
    all_file_path = []
    for row in all_file:
        if any([key_word in row["file_path"] for key_word in keywords]):
            
            if not uninclude_keywords:
                total += 1
                all_file_path.append(row["file_path"])
                continue
                
            
            if not any([key_word in row["file_path"] for key_word in uninclude_keywords]):
                total += 1
                all_file_path.append(row["file_path"])
                
    return total, all_file_path

# 寻找假阴性

In [15]:
# 全部文件中file_name包含关键字的文件
total, all_file_path = query_file_total_by_any_keywords(all_file, {"真题", "试卷", "考试", "答案"}, {"/千套简历模板+简历指导+面试真题-求职加薪必备", "申论", "/培训体系"})
total

6520

In [16]:
# 筛选出来的文件中file_name包含关键字的文件
total, all_classified_file_path = query_file_total_by_any_keywords(positive_file, {"真题", "试卷", "考试", "答案"})
total

6025

In [17]:
# 这里打印的预测为False，但是通过文件名检测是True的
set(all_file_path) - set(all_classified_file_path)

{'./data/docx/202305/10T电子书 超全/👑👑各类分类/学习资料/各种考试/2022一级建造师/独家-2021年一级建造师神级资料/法规/2021一建【法规】考前一小时【必看、重点推荐】.doc',
 './data/docx/202305/10T电子书 超全/👑👑各类分类/学习资料/各种考试/2022一级建造师/独家-2021年一级建造师神级资料/管理/2021一建【管理】大立押-考前点睛补充【重点推荐】.doc',
 './data/docx/202305/10T电子书 超全/👑👑各类分类/学习资料/各种考试/2022一级建造师/独家-2021年一级建造师神级资料/管理/2021一建【管理】考前一小时直播-管理最终版【推荐必看】.doc',
 './data/docx/202305/10T电子书 超全/👑👑各类分类/学习资料/各种考试/2022一级建造师/独家-2021年一级建造师神级资料/经济/2021一建【经济】大立押-考前点睛补充【重点推荐】.doc',
 './data/docx/202305/10T电子书 超全/👑👑各类分类/学习资料/各种考试/2022一级建造师/独家-2021年一级建造师神级资料/经济/2021一建【经济】考前一小时【重点推荐】.doc',
 './data/docx/202305/10T电子书 超全/👑👑各类分类/学习资料/各种考试/注册营养师资料-17.1GB/《中国居民膳食指南》折页/一日三餐带量营养食谱2000kcal.docx',
 './data/docx/202305/10T电子书 超全/👑👑各类分类/学习资料/各种考试/注册营养师资料-17.1GB/每日膳食营养素/中国各种常见食物营养成分表.doc',
 './data/docx/202305/10T电子书 超全/👑👑各类分类/学习资料/各种考试/注册营养师资料-17.1GB/每日膳食营养素/含膳食纤维最高的60种食物.doc',
 './data/docx/202305/10T电子书 超全/👑👑各类分类/学习资料/各种考试/注册营养师资料-17.1GB/注册营养师备考群电子版资料下载/指南标准/附件1.保健食品原料目录（一）(1).docx',
 './data/docx/202305/10T电子书 超全/👑👑各类分类/学习资料/各种考试/注册

# 寻找假阳性

In [18]:
# 查找以下关键字都不在路径中存在的
not_required_keywords = {
    "真题", 
    "试卷", 
    "考试", 
    "答案", 
    "高中", 
    "初中",
    "公考遴选", 
    "练习题", 
    "奥数", 
    "单元测试",
    "原卷",
    "习题",
    "笔试题",
    "试题",
    "解析版",
    "/练习",
    "题库"
}

total, all_file_path = query_file_total_by_any_keywords(all_file, {"/"}, not_required_keywords)
total

43155

In [19]:
total, all_classified_file_path = query_file_total_by_any_keywords(negative_file, {"/"}, not_required_keywords)
total

42649

In [20]:
# 从文件路径的关键字推则为False,但是预测是True
set(all_file_path) - set(all_classified_file_path)

{'./data/docx/202305/10T电子书 超全/👑👑各类分类/各类书籍/中国书法全集/中国书法全集简介.doc',
 './data/docx/202305/type/doc/11920527.doc',
 './data/docx/202307/10TB精整资料合集【公众号：锦技社】/｜ 精选电子书籍 ｜ @【公众号：锦技社】/_ 精选电子书籍合集 _/1.2G股票投资书籍/技术分析/Others/K线战法.doc',
 './data/docx/202307/10TB精整资料合集【公众号：锦技社】/｜ 精选电子书籍 ｜ @【公众号：锦技社】/_ 精选电子书籍合集 _/1.2G股票投资书籍/艾略特波浪/波动原理的标记方法.doc',
 './data/docx/202307/10TB精整资料合集【公众号：锦技社】/｜ 精选电子书籍 ｜ @【公众号：锦技社】/_ 精选电子书籍合集 _/掌盟☯️研报中心重磅9月29日更新【内部流出】/01掌盟☯️商业计划书与可行性报告大全/34、市场可行性报告/【公益知识库zscc.club】某镇厨具产业前景分析.doc',
 './data/docx/202307/10TB精整资料合集【公众号：锦技社】/｜ 精选电子书籍 ｜ @【公众号：锦技社】/_ 精选电子书籍合集 _/掌盟☯️研报中心重磅9月29日更新【内部流出】/01掌盟☯️商业计划书与可行性报告大全/34、市场可行性报告/【公益知识库zscc.club】某镇厨具产业前景分析_20210907221446.doc',
 './data/docx/202307/10TB精整资料合集【公众号：锦技社】/｜ 精选电子书籍 ｜ @【公众号：锦技社】/_ 精选电子书籍合集 _/掌盟☯️研报中心重磅9月29日更新【内部流出】/01掌盟☯️商业计划书与可行性报告大全/37、太阳能方面的计划书与报告/【公益知识库zscc.club】太阳能逆变器开发思路和方案.doc',
 './data/docx/202307/10TB精整资料合集【公众号：锦技社】/｜ 精选电子书籍 ｜ @【公众号：锦技社】/_ 精选电子书籍合集 _/掌盟☯️研报中心重磅9月29日更新【内部流出】/01掌盟☯️商业计划书与可行性报告大全/《商业计划》社群会员专属资料请勿外传（原盘）/15、市场可行性报告/某镇

In [39]:
import shutil
import os

move_files_pathe = [
  
]

for file in move_files_pathe:
    file_name = file.split("/")[-1]
    shutil.copy(os.path.join("../", file), os.path.join("./negative_file", file_name))